## Pipeline обработки запроса

### Загрузка данных и функций (достаточно выполнить 1 раз)

In [1]:
from main import setup_environment, load_data
from retriever import retriever, RetrieverOut
from planner import planner, PlannerOut
from grounder import grounder
from executor import executor

from openai import OpenAI
from config import PipelineConfig


api_key, db_path = setup_environment()
db = load_data(db_path, wave_filter=["2025-03"])    # Фильтр по последней волне

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
)

2025-11-12 12:40:00 | INFO     | main                 | Environment loaded: DB_PATH=C:\Users\kateu\Documents\Ivanovs Index\V2\DB\25W3\db_CORRECTED.parquet
2025-11-12 12:40:00 | INFO     | main                 | Loading data from C:\Users\kateu\Documents\Ivanovs Index\V2\DB\25W3\db_CORRECTED.parquet
2025-11-12 12:40:01 | INFO     | main                 | Full dataset shape: (26505646, 5)
2025-11-12 12:40:01 | INFO     | main                 | Filtered by waves: ['2025-03'], new shape: (672405, 5)


### Настройка используемых моделей и их температур

Если нужно поменять модель, это делается здесь

In [ ]:
PPL_cfg = PipelineConfig.setup(
    df=db, client=client,
    # параметры ретривера
    retriever_params={
        "model": "openrouter/polaris-alpha",
        # "model": "alibaba/tongyi-deepresearch-30b-a3b:free",
        "temperature": 0.4
    },
    # параметры планировщика
    planner_params={
        "model": "meta-llama/llama-3.3-70b-instruct:free",
        # !!! "model": "deepseek/deepseek-chat-v3.1", # Платная модель
        "temperature": 0.1
    }
)

### Еще варианты моделей
# "alibaba/tongyi-deepresearch-30b-a3b:free"
# "meta-llama/llama-4-maverick:free"
# !!! "deepseek/deepseek-chat-v3.1" # Платная модель

### Retriever

Извлекает релевантные вопросы на основе запроса пользователя и всего набора вопросов

Пользовательский запрос

In [3]:
user_query = "Я хочу оценить распределение потребительских предпочтений по сетям магазинов"

Обращение к LLM

> ! Возможны проблемы парсинга ответов

In [ ]:
# Сохранение, чтобы не делать 1 и тот же запрос кучу раз:
# можно 1 раз сохранить, а далее только читать

retriever_out = retriever(user_query, PPL_cfg)
retriever_out.save("cur_retrieved.json")

2025-11-12 12:40:10 | INFO     | retriever            | Starting retriever for query: Я хочу оценить распределение потребительских предпочтений по сетям магазинов...
2025-11-12 12:40:10 | DEBUG    | retriever            | Calling LLM with model: openrouter/polaris-alpha
2025-11-12 12:40:10 | DEBUG    | retriever            | Prompt length: 77091
2025-11-12 12:40:14 | INFO     | httpx                | HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-12 12:40:29 | DEBUG    | retriever            | LLM response length: 3173
2025-11-12 12:40:29 | DEBUG    | retriever            | Parsing retriever response
2025-11-12 12:40:29 | DEBUG    | retriever            | Successfully parsed 10 questions
2025-11-12 12:40:29 | INFO     | retriever            | Found 10 candidate questions
2025-11-12 12:40:29 | INFO     | retriever            | Retriever completed with 10 questions


In [5]:
retriever_out = RetrieverOut.load("cur_retrieved.json")
print(retriever_out)

1. [98/100] '[Q115] В каких магазинах Вы делаете покупки?'
	Reason: [Основной] Базовый вопрос для оценки охвата и предпочтений по сетям (многовыбор, даёт распределение долей покупателей по брендам).
2. [96/100] '[S8] В каком магазине Вы покупали основную часть продуктов в последнем месяце?'
	Reason: [Основной] Позволяет выделить основной магазин (primary store) и построить распределение «главного выбора» по сетям, а не только факт посещения.
3. [92/100] '[Q115_prev] В каких магазинах Вы делали покупки год назад?'
	Reason: [Основной / Контекстный] Даёт динамику предпочтений по сетям (перетоки клиентов), усиливает интерпретацию текущего распределения.
4. [88/100] '[Q115_online] Через какой сервис / у какой сети покупаете продукты питания ОНЛАЙН?'
	Reason: [Основной (для онлайн-сегмента)] Учитывает долю предпочтений по сетям в онлайн-канале, позволяя разделить офлайн vs онлайн предпочтения.
5. [78/100] '[Q114] Где Вы покупаете продукты питания? @ Небольшой магазин современного формата (об

### Planner

Строит план на основе пользовательского запроса и набора релевантных ответов, к которым подмешиваются их ответы

В план включаются команды из [`capability_spec.py`](./capability_spec.py)

In [11]:
# Сохранение, чтобы не делать 1 и тот же запрос кучу раз:
# можно 1 раз сохранить, а далее только читать

planner_out = planner(user_query, retriever_out, PPL_cfg)
planner_out.save("cur_planned.json")

2025-11-12 12:44:26 | INFO     | planner              | Starting planner for query: Я хочу оценить распределение потребительских предпочтений по сетям магазинов...
2025-11-12 12:44:26 | DEBUG    | planner              | Working with 10 questions
2025-11-12 12:44:26 | DEBUG    | planner              | Generated prompt of length: 9600
2025-11-12 12:44:26 | DEBUG    | planner              | Calling LLM with model: deepseek/deepseek-chat-v3.1
2025-11-12 12:44:28 | INFO     | httpx                | HTTP Request: POST https://openrouter.ai/api/v1/responses "HTTP/1.1 200 OK"
2025-11-12 12:44:32 | DEBUG    | planner              | Plan received: 6 steps
2025-11-12 12:44:32 | DEBUG    | planner              | Plan analysis: Для оценки распределения потребительских предпочтений по сетям магазинов наиболее релевантными являются вопросы о текущих покупках ([Q115]), основном месте покупок ([S8]) и доступности магазинов ([M5]). План включает загрузку данных, анализ распределения по этим ключевым воп

In [12]:
planner_out = PlannerOut.load("cur_planned.json")
print(planner_out)

АНАЛИЗ: Для оценки распределения потребительских предпочтений по сетям магазинов наиболее релевантными являются вопросы о текущих покупках ([Q115]), основном месте покупок ([S8]) и доступности магазинов ([M5]). План включает загрузку данных, анализ распределения по этим ключевым вопросам и дополнительный анализ по доходу для сегментации.
0. [s1] OperationType.LOAD_DATA
	Goal: Загрузить данные опроса за март 2025 года
	Inputs: {'waves': ['2025-03']}
	Outputs: ['dataset']
	Constraints: {}
	Depends on: []
1. [s2] OperationType.PIVOT
	Goal: Проанализировать распределение ответов на вопрос о том, в каких магазинах респонденты делают покупки
	Inputs: {'dataset': 'dataset', 'question': '[Q115] В каких магазинах Вы делаете покупки?'}
	Outputs: ['pivot_q115']
	Constraints: {'question': 'MUST be from allowed_questions'}
	Depends on: ['s1']
2. [s3] OperationType.PIVOT
	Goal: Проанализировать распределение ответов на вопрос о магазине, где покупалась основная часть продуктов
	Inputs: {'dataset': '

### Grounder

Привязка шагов плана к имеющимся функциям (из [`operations.py`](./operations.py))

In [13]:
grounder_out = grounder(planner_out)

2025-11-12 12:44:41 | INFO     | grounder             | Starting grounder
2025-11-12 12:44:41 | DEBUG    | grounder             | Grounding step 1/6: s1
2025-11-12 12:44:41 | DEBUG    | grounder             | Successfully grounded step s1: LOAD_DATA
2025-11-12 12:44:41 | DEBUG    | grounder             | Grounding step 2/6: s2
2025-11-12 12:44:41 | DEBUG    | grounder             | Successfully grounded step s2: PIVOT
2025-11-12 12:44:41 | DEBUG    | grounder             | Grounding step 3/6: s3
2025-11-12 12:44:41 | DEBUG    | grounder             | Successfully grounded step s3: PIVOT
2025-11-12 12:44:41 | DEBUG    | grounder             | Grounding step 4/6: s4
2025-11-12 12:44:41 | DEBUG    | grounder             | Successfully grounded step s4: PIVOT
2025-11-12 12:44:41 | DEBUG    | grounder             | Grounding step 5/6: s5
2025-11-12 12:44:41 | DEBUG    | grounder             | Successfully grounded step s5: FILTER
2025-11-12 12:44:41 | DEBUG    | grounder             | Groun

### Executor

Валидирует план, осуществляет топологическую сортировку, выполняет последовательность шагов

In [14]:
ctx = {"dataset": db}
ctx = executor(grounder_out, ctx)

2025-11-12 12:44:43 | INFO     | executor             | Starting executor with 6 steps
2025-11-12 12:44:43 | DEBUG    | executor             | Validating step dependencies
2025-11-12 12:44:43 | DEBUG    | executor             | Performing topological sort
2025-11-12 12:44:43 | DEBUG    | executor             | Topological sort completed: ['s1', 's2', 's3', 's4', 's5', 's6']
2025-11-12 12:44:43 | DEBUG    | executor             | Dependencies validation passed
2025-11-12 12:44:43 | DEBUG    | executor             | Performing topological sort
2025-11-12 12:44:43 | DEBUG    | executor             | Topological sort completed: ['s1', 's2', 's3', 's4', 's5', 's6']
2025-11-12 12:44:43 | INFO     | executor             | Steps execution order: ['s1', 's2', 's3', 's4', 's5', 's6']
2025-11-12 12:44:43 | INFO     | executor             | Executing step 1/6: s1 (LOAD_DATA)
2025-11-12 12:44:43 | DEBUG    | executor             |   waves = ['2025-03'] (literal)
2025-11-12 12:44:43 | DEBUG    | exe

Далее можно извлекать созданные таблицы из `ctx`

In [19]:
opts = [f"{i}. '{n}'" for i, n in enumerate(ctx.keys(), start=1)]
opts = "\n".join(opts)

print(f"Доступные варианты:\n{opts}")

Доступные варианты:
1. 'dataset'
2. 'pivot'
3. 'pivot_q115'
4. 'pivot_s8'
5. 'pivot_m5'
6. 'filtered_dataset'
7. 'filtered_dataset_high_income'
8. 'pivot_q115_high_income'


Пока что извлекать и смотреть можно только ручками...

In [ ]:
ctx["pivot"]